In [60]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

import warnings
from numpy import VisibleDeprecationWarning
warnings.filterwarnings(action = "ignore", category = VisibleDeprecationWarning)


from torch import tensor
from torch.autograd import Variable
from generate_data import generate_data, data_loader

In [61]:
batch_size = 4

lr = 0.00005

loss_fn = nn.BCELoss()

In [62]:
class RNN_predictor(nn.Module):
    def __init__(self, hidden_dim):
        super(RNN_predictor, self).__init__()
        
        # self.hidden_dim = hidden_dim
        
        # self.layer_dim = layer_dim
        
        self.rnn = nn.RNN(3, hidden_dim, num_layers = 8,nonlinearity='relu')
        
        self.fc = nn.Linear(hidden_dim, 1)
    
        self.activation = nn.Softmax(0)
        
    def forward(self, x):
        
        # Initialize hidden state with zeros
        # hidden = Variable(torch.zeros(self.layer_dim, x.shape[1], self.hidden_dim))

        # One time step
        out, hn = self.rnn(x)
        out = self.fc(out[ -1])
        
        return self.activation(out)
        # return out[-1]


model = RNN_predictor(5)

In [63]:
print(model)

RNN_predictor(
  (rnn): RNN(3, 5, num_layers=8)
  (fc): Linear(in_features=5, out_features=1, bias=True)
  (activation): Softmax(dim=0)
)


In [58]:
def test(model, X_test, y_test, min_length):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=batch_size, min_length = min_length)
    acc = 0.0
    i = 0
    F1 = 0
    f1a, f1b = [], []
    for data, labels in test_batches:
        # pred = model(data).view((-1,1))
        # print(pred.shape)
        pred = model(data)
        a, b = torch.round(pred), torch.argmax(labels,1)
        print(a.shape, b.shape)

        corr = (a == b).detach().float().mean().item()
        acc += corr
        f1a.append(a)
        f1b.append(b)
        i += 1
    F1 = f1_score(torch.cat(f1a), torch.cat(f1b))
    return acc/i, F1/i

def train(model,n_epochs,lr, min_length):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()

    losss, accs, f1s = [], [], []
    pe = 10
    for i in range(n_epochs):
        model.train()
        X_train, y_train = generate_data("train", type = 2)
        X_validation, y_validation = generate_data("validation", type = 2)
        train_batches = data_loader(X_train,y_train,batch_size=batch_size, min_length = min_length)

        sum_loss = 0.0
        i_loss = 0

        
        for data, labels in train_batches:
            pred = model(data)
            # loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred, torch.argmax(labels,1).type(torch.FloatTensor))
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1
        
        if i%pe == 0:
            losss.append(sum_loss/i_loss)
            t, F1 = test(model,X_train, y_train, 20)
            accs.append(t)
            f1s.append(F1)

            print(f"epoch {i}/{n_epochs} - loss {sum_loss/i_loss:.3} - acc {t:.3} - F1 {F1:.3}")

    return np.array(losss), np.array(accs), np.array(f1s)


In [59]:
def train_and_test(model, lr):
    N = 4*int(1e4)

    min_length = 20

   
    loss, acc, f1s = train(model, 2000, lr, min_length = 20)
    
    min_length = 50

    X_test, y_test = generate_data("test", type = 2)

    t, f1 = test(model, X_test, y_test, min_length = 50)
    
    print(f"Final acc {t:.3} - F1 {f1}")

    return loss, acc, t, f1


lrs = [0.1,0.0005, 0.001]
sizes = [2, 4, 8]


save_i = 0
for lr in lrs:
    for size in sizes:
        losss, accs, faccs, ff1s = [], [], [], []
        data = {"lr":lr,'size':size}

        for i in range(3):
            model = RNN_predictor(size)
            print("Training with lr", lr,"and size", size)
            loss, acc, facc, ff1 = train_and_test(model, lr)
            losss.append(loss)
            accs.append(acc)
            faccs.append(facc) 
            ff1s.append(ff1) 

        save_i += 1
        data['loss'] = np.array(losss)
        data['acc'] = np.array(accs)
        data['facc'] = np.array(faccs)
        data['ff1'] = np.array(ff1s)

        np.save(f"data{save_i}.npy",data, allow_pickle=True)

        



Training with lr 0.1 and size 2
torch.Size([4, 1]) torch.Size([4])
torch.Size([4, 1]) torch.Size([4])
torch.Size([4, 1]) torch.Size([4])
torch.Size([4, 1]) torch.Size([4])
torch.Size([4, 1]) torch.Size([4])


c:\Users\jakob\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.